In [1]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
from utils import iter_df_read, get_array_diff
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn")
sns.set(font_scale=1.5)

# Modificar por ruta local de almacenamiento de datos
BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Atributos primera iteración
- **Notas de colegio**: Egresados Colegio, `PROM_NOTAS_ALU`
- **Región del colegio**: Egresados Colegio, `COD_REG_RBD`
- **Modalidad del colegio (científico humanista / técnico)**: Egresados Colegio, `COD_ENSE`
- **Tipo de establecimiento (particular, subvencionado, municipal)**: Establecimientos, `cod_depe`
- **Género**: Matriculados, `gen_alu`.
- **Edad**: Matriculados, `fec_nac_alu`.
- **Área del conocimiento de la carrera**: Matriculados, `area_conocimiento`.
- **Región de la institución de educación superior**: Matriculados, `region_sede`.

# Lectura datos

## Desertores

In [2]:
desertores_1 = pd.read_csv(f"{BASE}/tmp_data/desertores_1.csv.gz")
desertores_1.head()

,cat_periodo,codigo_unico,mrun,desertor_1
0,2015,I374S1C9J1V1,5.0,0
1,2015,I498S6C132J2V1,37.0,0
2,2015,I111S14C309J2V1,118.0,0
3,2015,I260S44C120J2V1,184.0,0
4,2015,I106S3C64J4V1,253.0,1


## Matrículas

In [3]:
cols_mat = [
    'cat_periodo', 'codigo_unico', 'mrun', 'tipo_inst_1', 
    'gen_alu', 'fec_nac_alu', 'area_conocimiento', 'region_sede',
    "anio_ing_carr_ori"
]

instituciones = ['Institutos Profesionales', 'Centros de Formación Técnica']

matriculados = pd.concat([
    iter_df_read(f, "tipo_inst_1", instituciones, sep=";", usecols=cols_mat)
    for f in tqdm(
        glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"),
        total=8
    )
]).dropna(subset="mrun").drop_duplicates().reset_index(drop=True)

matriculados = matriculados[
    (matriculados["anio_ing_carr_ori"] >= 2015)
    & (matriculados["anio_ing_carr_ori"] < 2022)
]

print(f"Cantidad de registros: {matriculados.shape[0]}\n")
print(f"Cantidad de nulos: \n{matriculados.isna().sum()}")
matriculados.head()

  0%|          | 0/8 [00:00<?, ?it/s]

Cantidad de registros: 3202225

Cantidad de nulos: 
cat_periodo          0
codigo_unico         0
mrun                 0
gen_alu              0
fec_nac_alu          0
anio_ing_carr_ori    0
tipo_inst_1          0
region_sede          0
area_conocimiento    0
dtype: int64


,cat_periodo,codigo_unico,mrun,gen_alu,fec_nac_alu,anio_ing_carr_ori,tipo_inst_1,region_sede,area_conocimiento
0,2015,I374S1C9J1V1,5.0,2,199105,2015,Centros de Formación Técnica,Metropolitana,Salud
1,2015,I498S6C132J2V1,37.0,1,197808,2015,Centros de Formación Técnica,Biobío,Tecnología
3,2015,I111S14C309J2V1,118.0,1,199102,2015,Institutos Profesionales,Metropolitana,Administración y Comercio
6,2015,I106S3C64J4V1,253.0,1,198404,2015,Institutos Profesionales,Metropolitana,Administración y Comercio
8,2015,I221S1C57J1V1,370.0,2,198910,2015,Centros de Formación Técnica,Metropolitana,Administración y Comercio


In [4]:
mat_des = pd.merge(
    matriculados,
    desertores_1,
    how="left"
)

matriculados.shape, desertores_1.shape, mat_des.shape

((3202225, 9), (3012141, 4), (3202225, 10))

In [5]:
mat_des[mat_des["desertor_1"].isna()]["cat_periodo"].value_counts()

2022    332793
Name: cat_periodo, dtype: int64

In [6]:
mat_des = mat_des.dropna().reset_index(drop=True)

## Colegios

### Egresados EM

In [7]:
cols_egreso_col = [
    'MRUN', 'PROM_NOTAS_ALU', 'COD_REG_RBD', 'RBD', 'COD_ENSE', 'COD_GRADO', "AGNO"
]

egreso_colegio = pd.concat([
    iter_df_read(f, "MRUN", mat_des["mrun"].unique(), sep=";", usecols=cols_egreso_col)
    for f in tqdm(
        glob(f"{BASE}raw_data/egresados_colegios/20220325_Notas_y_Egresados_Enseñanza_Media_*.csv"),
        total=20
    )
]).dropna(subset="MRUN").drop_duplicates().reset_index(drop=True).rename(
    columns={"MRUN": "mrun"}
).sort_values(
    by=["COD_GRADO"], ascending=False
).drop_duplicates(
    subset=["mrun"], keep="first"
).reset_index(drop=True)

print(f"Cantidad de registros: {egreso_colegio.shape[0]}\n")
print(f"Cantidad de nulos: \n{egreso_colegio.isna().sum()}")
egreso_colegio.head()

  0%|          | 0/20 [00:00<?, ?it/s]

Cantidad de registros: 1048218

Cantidad de nulos: 
AGNO              0
RBD               0
COD_REG_RBD       0
COD_ENSE          0
COD_GRADO         0
mrun              0
PROM_NOTAS_ALU    0
dtype: int64


,AGNO,RBD,COD_REG_RBD,COD_ENSE,COD_GRADO,mrun,PROM_NOTAS_ALU
0,2017,10780,13,363,4,25581603,"6,3"
1,2018,12332,5,510,4,14294511,"6,4"
2,2015,16634,7,710,4,5928282,"6,1"
3,2008,6643,9,610,4,24845591,"5,6"
4,2008,9957,13,510,4,24844652,"5,6"


### Establecimientos
https://datosabiertos.mineduc.cl/directorio-de-establecimientos-educacionales/

In [9]:
establecimientos_04_21 = pd.concat([
    pd.read_csv(f, sep=";").rename(columns={
        "RBD":"rbd",
        "COD_DEPE":"cod_depe",
        "AGNO": "agno",
        "ïagno": "agno"
    })
    for f in tqdm(
        glob(f"{BASE}raw_data/directorio_colegios/Directorio_*.csv"),
        total=18
    )
])[[
    "rbd", "cod_depe", "agno"
]].dropna()

establecimientos_04_21 = establecimientos_04_21[
    establecimientos_04_21["rbd"].isin(egreso_colegio["RBD"].unique())
].drop_duplicates().reset_index(drop=True)

print(f"Cantidad de registros: {establecimientos_04_21.shape[0]}\n")
print(f"Cantidad de nulos: \n{establecimientos_04_21.isna().sum()}")
establecimientos_04_21.head()

  0%|          | 0/18 [00:00<?, ?it/s]

Cantidad de registros: 48786

Cantidad de nulos: 
rbd         0
cod_depe    0
agno        0
dtype: int64


,rbd,cod_depe,agno
0,1.0,2.0,2004.0
1,3.0,2.0,2004.0
2,4.0,2.0,2004.0
3,5.0,2.0,2004.0
4,7.0,2.0,2004.0


In [10]:
colegios = pd.merge(
    egreso_colegio.rename(columns={
        "RBD":"rbd",
        "AGNO": "agno"
    }),
    establecimientos_04_21,
    how="left"
).drop_duplicates().reset_index(drop=True)

colegios.shape, egreso_colegio.shape

((1048218, 8), (1048218, 7))

In [11]:
colegios.head()

,agno,rbd,COD_REG_RBD,COD_ENSE,COD_GRADO,mrun,PROM_NOTAS_ALU,cod_depe
0,2017,10780,13,363,4,25581603,"6,3",NaN
1,2018,12332,5,510,4,14294511,"6,4",NaN
2,2015,16634,7,710,4,5928282,"6,1",3.0
3,2008,6643,9,610,4,24845591,"5,6",3.0
4,2008,9957,13,510,4,24844652,"5,6",3.0


## Merge matriz 1

In [13]:
matriz_1 = pd.merge(
    mat_des,
    colegios[["mrun", "COD_REG_RBD", "COD_ENSE", "PROM_NOTAS_ALU", "cod_depe"]].rename(columns={
        "COD_REG_RBD":"region_colegio",
        "COD_ENSE": "tipo_ensenanza_colegio", #CH, tecnico, etc
        "PROM_NOTAS_ALU": "prom_notas_ultimo_anio_col",
        "cod_depe": "dependencia_colegio" #municipal, subvencionado, etc
    }),
    how="left"
).drop_duplicates().reset_index(drop=True)

colegios.shape, mat_des.shape, matriz_1.shape

((1048218, 8), (2869432, 10), (2869432, 14))

In [14]:
matriz_1.isna().sum()

cat_periodo                         0
codigo_unico                        0
mrun                                0
gen_alu                             0
fec_nac_alu                         0
anio_ing_carr_ori                   0
tipo_inst_1                         0
region_sede                         0
area_conocimiento                   0
desertor_1                          0
region_colegio                 365341
tipo_ensenanza_colegio         365341
prom_notas_ultimo_anio_col     365341
dependencia_colegio           1010032
dtype: int64

In [15]:
matriz_1.to_csv(f"{BASE}work_data/matriz_desercion_1.csv.gz", index=False, compression="gzip")